In [1]:
!pip install diffusers transformers hf_transfer

In [2]:
!pip install accelerate==0.33.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 6.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found 

In [3]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [4]:
import torch
from diffusers import AutoencoderKLCogVideoX, CogVideoXPipeline, CogVideoXTransformer3DModel
from diffusers.utils import export_to_video
from transformers import T5EncoderModel

2025-04-10 21:16:17.200969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744319777.633318      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744319777.744513      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Models: "THUDM/CogVideoX-2b" or "THUDM/CogVideoX-5b"
model_id = "THUDM/CogVideoX-5b"

In [6]:
transformer = CogVideoXTransformer3DModel.from_pretrained("camenduru/cogvideox-5b-float16", subfolder="transformer", torch_dtype=torch.float16)
text_encoder = T5EncoderModel.from_pretrained("camenduru/cogvideox-5b-float16", subfolder="text_encoder", torch_dtype=torch.float16)
vae = AutoencoderKLCogVideoX.from_pretrained(model_id, subfolder="vae", torch_dtype=torch.float16)

config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

(…)ion_pytorch_model.safetensors.index.json:   0%|          | 0.00/103k [00:00<?, ?B/s]

(…)pytorch_model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

(…)pytorch_model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

(…)pytorch_model-00003-of-00003.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/872 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/862M [00:00<?, ?B/s]

In [7]:
pipe = CogVideoXPipeline.from_pretrained(
    model_id,
    text_encoder=text_encoder,
    transformer=transformer,
    vae=vae,
    torch_dtype=torch.float16,
)

model_index.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
pipe.enable_sequential_cpu_offload()
# pipe.vae.enable_tiling()

In [9]:
prompt = (
    "An electron, glowing with a faint blue hue, orbits serenely around a shimmering atomic nucleus in a vast quantum field."
"The electron emits subtle pulses of energy, creating soft, rhythmic waves that ripple through the surrounding space. Nearby, other subatomic particles gather, drawn in by the electron's mesmerizing oscillations, some resonating in sync."
"Streams of radiant energy flow like currents through the field, casting an ethereal glow that dances across the quantum landscape. The electron's motion is elegant and deliberate, reflecting both precision and harmony."
"The background shimmers with probability clouds and vibrant bursts of light, enhancing the mysterious and almost magical nature of this atomic ballet."
)

In [10]:
video = pipe(prompt=prompt, guidance_scale=6, use_dynamic_cfg=True, num_inference_steps=5).frames[0]

  0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 676.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 182.12 MiB is free. Process 2356 has 14.56 GiB memory in use. Of the allocated memory 14.13 GiB is allocated by PyTorch, and 308.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
export_to_video(video, "output.mp4", fps=8)

In [10]:
import torch
import nltk
import numpy as np
import os
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from PIL import Image
import cv2
from tqdm import tqdm

Loading pipeline...


model_index.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/391M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.53G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/6.10G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

['The Pythagorean Theorem is one of the fundamental principles in geometry.', 'It describes a relationship between the sides of a right triangle.', 'A right triangle is a triangle that contains a 90-degree angle.', 'In this theorem, the three sides of the triangle are referred to as the two legs and the hypotenuse.', 'The two legs are the sides that form the right angle, while the hypotenuse is the longest side, opposite the right angle.', 'The Pythagorean Theorem states that the square of the length of the hypotenuse is equal to the sum of the squares of the lengths of the other two sides.', 'Mathematically, this is expressed as: a² + b² = c² Here, a and b represent the lengths of the two legs, and c represents the length of the hypotenuse.', 'This theorem only applies to right triangles.', 'If a triangle does not have a right angle, the Pythagorean Theorem does not apply.', 'To visualize this, imagine a right triangle with a square drawn on each of its sides.', 'The area of the squar

['The Pythagorean Theorem is one of the fundamental principles in geometry.', 'It describes a relationship between the sides of a right triangle.', 'A right triangle is a triangle that contains a 90-degree angle.', 'In this theorem, the three sides of the triangle are referred to as the two legs and the hypotenuse.', 'The two legs are the sides that form the right angle, while the hypotenuse is the longest side, opposite the right angle.', 'The Pythagorean Theorem states that the square of the length of the hypotenuse is equal to the sum of the squares of the lengths of the other two sides.', 'Mathematically, this is expressed as: a² + b² = c² Here, a and b represent the lengths of the two legs, and c represents the length of the hypotenuse.', 'This theorem only applies to right triangles.', 'If a triangle does not have a right angle, the Pythagorean Theorem does not apply.', 'To visualize this, imagine a right triangle with a square drawn on each of its sides.', 'The area of the squar

In [14]:
import torch
import nltk
import numpy as np
import os
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from PIL import Image
import cv2
from tqdm import tqdm

# Initialize NLTK for text processing
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the pipeline
print("Loading pipeline...")
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)

# Original prompt
prompt = """
The Pythagorean Theorem is one of the fundamental principles in geometry. It describes a relationship between the sides of a right triangle. A right triangle is a triangle that contains a 90-degree angle. In this theorem, the three sides of the triangle are referred to as the two legs and the hypotenuse. The two legs are the sides that form the right angle, while the hypotenuse is the longest side, opposite the right angle. The Pythagorean Theorem states that the square of the length of the hypotenuse is equal to the sum of the squares of the lengths of the other two sides.

Mathematically, this is expressed as:
a² + b² = c²

Here, a and b represent the lengths of the two legs, and c represents the length of the hypotenuse. This theorem only applies to right triangles. If a triangle does not have a right angle, the Pythagorean Theorem does not apply. To visualize this, imagine a right triangle with a square drawn on each of its sides. The area of the square on the hypotenuse will equal the combined area of the squares on the other two sides. This is the essence of the Pythagorean Theorem.

The Pythagorean Theorem was named after the ancient Greek mathematician Pythagoras. However, the knowledge of this relationship predates Pythagoras, with evidence suggesting that ancient civilizations such as the Babylonians and Egyptians were aware of the principle. Pythagoras, however, is credited with being the first to formally prove it. Over time, the theorem became a cornerstone of Euclidean geometry.

In practical terms, the Pythagorean Theorem is incredibly useful. It helps in calculating distances, heights, and angles in various fields such as engineering, architecture, astronomy, and navigation. For example, if you know the lengths of the legs of a right triangle, you can easily find the length of the hypotenuse by applying the Pythagorean Theorem. Conversely, if you know the hypotenuse and one leg, you can find the missing leg.

Let’s consider a simple example: imagine a right triangle where one leg is 3 units long and the other leg is 4 units long. To find the length of the hypotenuse, you can apply the Pythagorean Theorem:
a² + b² = c²
3² + 4² = c²
9 + 16 = c²
25 = c²
Taking the square root of both sides:
√25 = c
So, c = 5.
Therefore, the hypotenuse of this right triangle is 5 units long.

This is a practical demonstration of the Pythagorean Theorem in action. The theorem is not limited to simple integer values, though. It applies to all right triangles, regardless of the specific lengths of the sides, as long as the triangle remains a right triangle. Another example might be a triangle with legs of 5 and 12 units. Using the same steps:
5² + 12² = c²
25 + 144 = c²
169 = c²
√169 = c
c = 13.
Thus, the hypotenuse in this case would be 13 units long.

The Pythagorean Theorem is also helpful in real-world applications, such as finding the distance between two points on a coordinate plane. Suppose you need to calculate the straight-line distance between two points, (x₁, y₁) and (x₂, y₂), on a grid. The horizontal distance between the points is |x₂ - x₁| and the vertical distance is |y₂ - y₁|. These can be treated as the two legs of a right triangle, and the distance between the points is the hypotenuse. The formula in this case becomes:
Distance = √[(x₂ - x₁)² + (y₂ - y₁)²]
This is directly derived from the Pythagorean Theorem.

While the Pythagorean Theorem is a powerful tool, there are some important points to consider. First, it only applies to right triangles. Second, the theorem involves squares and square roots, so it requires a basic understanding of algebra to use effectively. Fortunately, with practice, these concepts become easier to apply.

Additionally, the Pythagorean Theorem is a foundational principle in trigonometry, where it’s used to define other relationships, such as the sine, cosine, and tangent functions. These functions help describe angles and sides in right triangles and are used extensively in various scientific and engineering disciplines.

The Pythagorean Theorem also leads to interesting properties and extensions. For instance, in three-dimensional space, there is a generalized version of the theorem called the distance formula, which applies to right-angled triangles in 3D geometry. In this case, the formula is:
Distance = √[(x₂ - x₁)² + (y₂ - y₁)² + (z₂ - z₁)²]

Moreover, the Pythagorean Theorem has inspired countless mathematical discoveries and theorems. One notable result is the Pythagorean triple, where the lengths of the three sides of a right triangle are all integers. An example of a Pythagorean triple is the set {3, 4, 5}, where 3² + 4² = 5².

Another interesting extension of the Pythagorean Theorem is the concept of irrational numbers. In some cases, the length of the hypotenuse of a right triangle is not an integer, but an irrational number, such as the square root of 2. This was discovered by ancient Greek mathematicians and marked a major shift in the understanding of numbers.

In summary, the Pythagorean Theorem is one of the most important principles in mathematics. It provides a simple yet powerful way to relate the sides of a right triangle. From ancient times to modern applications, the Pythagorean Theorem has been essential in solving practical problems. Its impact extends beyond geometry into various fields like physics, engineering, and computer science. Understanding this theorem is not only important for students learning geometry but also for anyone seeking to understand the world of mathematical relationships and spatial reasoning.
"""

# Clean the prompt - remove extra whitespace and line breaks
clean_prompt = ' '.join(prompt.split())

# Break the prompt into sentences
sentences = nltk.sent_tokenize(clean_prompt)
print(sentences)

# If we have fewer than 30 sentences, we'll repeat or interpolate
if len(sentences) < 30:
    # Create interpolated prompts between sentences
    expanded_prompts = []
    for i in range(len(sentences)-1):
        current = sentences[i]
        next_sent = sentences[i+1]
        
        # Add the current sentence
        expanded_prompts.append(current)
        
        # If we need more granularity, interpolate between sentences
        if len(sentences) < 15:  # If very few sentences, add intermediate steps
            expanded_prompts.append(f"{current} {next_sent}")
    
    # Add the final sentence
    expanded_prompts.append(sentences[-1])
    
    # If we still need more, duplicate with slight variations
    if len(expanded_prompts) < 30:
        final_prompts = []
        repeat_factor = max(1, 30 // len(expanded_prompts))
        
        for prompt in expanded_prompts:
            for i in range(repeat_factor):
                if i == 0:
                    final_prompts.append(prompt)
                else:
                    # Add small variations for repeated prompts
                    intensity = ["gradually", "slowly", "steadily", "continually"][i % 4]
                    final_prompts.append(f"{intensity} {prompt}")
        
        # Take the first 30 or pad if needed
        if len(final_prompts) > 30:
            prompts = final_prompts[:30]
        else:
            # Pad with repeats of the last sentence if needed
            padding = 30 - len(final_prompts)
            prompts = final_prompts + [sentences[-1]] * padding
    else:
        prompts = expanded_prompts[:30]
else:
    # If we have enough sentences, take the first 30
    prompts = sentences[:30]

print(prompts)

print(f"Created {len(prompts)} sequential prompts")

# Parameters
num_frames_per_segment = 8  # Standard 8 frames per segment
fps = 8  # Standard fps
num_inference_steps = 10  # Quality setting

# Create temporary directory for frames
os.makedirs("temp_frames", exist_ok=True)

# Generate segments sequentially
all_frames = []
last_frame = None

print("Generating video segments:")
for i, segment_prompt in enumerate(tqdm(prompts)):
    print(f"\nSegment {i+1}/30: {segment_prompt[:50]}...")
    
    # Generate current segment
    with torch.no_grad():
        if last_frame is not None:
            # Use the last frame as conditioning (not all models support this)
            try:
                video_frames = pipe(
                    segment_prompt, 
                    num_inference_steps=num_inference_steps,
                    num_frames=num_frames_per_segment,
                    # Some models support conditioning with an image
                    # If your model doesn't support this, remove the next line
                    # image=last_frame
                ).frames
            except Exception as e:
                print(f"Image conditioning failed: {e}")
                # Fallback without image conditioning
                video_frames = pipe(
                    segment_prompt, 
                    num_inference_steps=num_inference_steps,
                    num_frames=num_frames_per_segment
                ).frames
        else:
            # First segment without conditioning
            video_frames = pipe(
                segment_prompt, 
                num_inference_steps=num_inference_steps,
                num_frames=num_frames_per_segment
            ).frames
    
    # Make sure we have the right format
    if len(video_frames.shape) == 5:  # If shape is [batch, frames, height, width, channels]
        video_frames = video_frames[0]  # Take the first batch
    
    # Store the last frame for next iteration
    last_frame = video_frames[-1]  # Last frame for continuation
    
    # Add to our collection
    all_frames.extend(video_frames)
    
    # Clear GPU memory
    torch.cuda.empty_cache()

# Convert frames to proper format for export
all_frames = torch.stack(all_frames) if isinstance(all_frames[0], torch.Tensor) else np.array(all_frames)

# Save the final video
output_path = "p_30sec.mp4"
export_to_video(all_frames, output_path, fps=fps)
print(f"Video saved to: {output_path}")

# Clean up
import shutil
if os.path.exists("temp_frames"):
    shutil.rmtree("temp_frames")

# Display information about the generated video
print(f"Generated video details:")
print(f"- Duration: {len(all_frames)/fps:.2f} seconds")
print(f"- Frames: {len(all_frames)}")
print(f"- FPS: {fps}")
print(f"- Resolution: {all_frames[0].shape[0]}x{all_frames[0].shape[1]}")

Loading pipeline...


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
import torch
import nltk
import numpy as np
import os
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler, StableVideoDiffusionPipeline
from diffusers.utils import export_to_video
from PIL import Image
import cv2
from tqdm import tqdm
import gc

# Set environment variable to help with memory allocation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Initialize NLTK for text processing
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Function to clear GPU memory
def clear_gpu_memory():
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.synchronize()

# Create a list of visual prompts for the Pythagorean theorem
visual_prompts = [
    "A right triangle with squares drawn on each side, demonstrating the Pythagorean theorem, simple diagram",
    "A visual representation of a² + b² = c², geometric demonstration",
    "Ancient Greek mathematician Pythagoras teaching geometry",
    "Mathematical diagram showing the Pythagorean theorem",
    "A right triangle with sides 3, 4, and 5 units",
    "Step-by-step calculation showing 3² + 4² = 5²",
    "A right triangle with sides 5, 12, and 13",
    "Coordinate plane showing distance calculation using Pythagorean theorem",
    "Visual representation of the distance formula",
    "Mathematical diagram of right triangles",
    "Trigonometric functions using right triangles",
    "3D visualization of the Pythagorean theorem",
    "Visual explanation of the distance formula in 3D space",
    "Pythagorean triples displayed as right triangles",
    "Visual proof using a right triangle with unit sides",
    "Development of the Pythagorean theorem across civilizations",
    "Applications of the Pythagorean theorem in architecture",
    "Students learning the Pythagorean theorem",
    "Computer graphics showing Pythagorean calculations",
    "Engineering applications of the Pythagorean theorem",
]

# Ensure we have enough visual prompts
while len(visual_prompts) < 30:
    visual_prompts.extend(visual_prompts[:30-len(visual_prompts)])

# Take the first 30 visual prompts
visual_prompts = visual_prompts[:30]

print(f"Created {len(visual_prompts)} visual prompts for video generation")

# Parameters for video generation - reduced for memory efficiency
num_frames_per_segment = 4  # Reduced from 8 to 4
fps = 8
num_inference_steps = 20  # Reduced from 25
height = 384  # Reduced from 512
width = 384   # Reduced from 512

# Create output directory for frames
os.makedirs("output_frames", exist_ok=True)

# Memory-efficient approach: Generate one segment at a time and save frames
all_frame_paths = []
segment_count = 0

print("Generating segments one by one...")

try:
    # Load text-to-image model first
    print("Loading text-to-image pipeline...")
    t2i_pipe = DiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5", 
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )
    t2i_pipe = t2i_pipe.to(device)
    
    # Generate the initial image
    print("Generating initial image...")
    init_image = t2i_pipe(
        prompt=visual_prompts[0],
        height=height,
        width=width,
        num_inference_steps=20  # Reduced for memory
    ).images[0]
    
    # Save the initial image
    init_image.save("initial_frame.jpg")
    
    # Unload the text-to-image model to free memory
    del t2i_pipe
    clear_gpu_memory()
    
    # Load image-to-video model
    print("Loading image-to-video pipeline...")
    i2v_pipe = StableVideoDiffusionPipeline.from_pretrained(
        "stabilityai/stable-video-diffusion-img2vid-xt", 
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )
    i2v_pipe.enable_model_cpu_offload()  # Enable CPU offloading to save GPU memory
    
    # Start with the initial image
    current_image = init_image
    
    # Process each segment
    for i, visual_prompt in enumerate(visual_prompts):
        print(f"\nSegment {i+1}/{len(visual_prompts)}: {visual_prompt[:50]}...")
        
        # Set seed for consistent but progressive generation
        generator = torch.Generator(device=device).manual_seed(42 + i)
        
        # Generate video segment
        outputs = i2v_pipe(
            image=current_image,
            num_frames=num_frames_per_segment,
            num_inference_steps=num_inference_steps,
            generator=generator,
            motion_bucket_id=min(100 + i*5, 255),  # Lower motion for memory efficiency
            fps=fps
        )
        
        # Get frames and save them individually
        frames = outputs.frames
        
        # Save each frame as an image file
        segment_frame_paths = []
        for f_idx, frame in enumerate(frames):
            frame_path = f"output_frames/segment_{i+1:02d}_frame_{f_idx+1:02d}.jpg"
            if isinstance(frame, torch.Tensor):
                # Convert tensor to PIL Image if needed
                if frame.dim() == 4:  # [batch, channels, height, width]
                    frame = frame.squeeze(0)
                if frame.shape[0] == 3:  # [channels, height, width]
                    frame = frame.permute(1, 2, 0)
                if frame.max() <= 1.0:
                    frame = (frame * 255).byte()
                frame = Image.fromarray(frame.cpu().numpy())
            
            # Save the frame
            frame.save(frame_path)
            segment_frame_paths.append(frame_path)
        
        # Update current image to the last frame
        current_image = Image.open(segment_frame_paths[-1])
        
        # Add frame paths to our collection
        all_frame_paths.extend(segment_frame_paths)
        
        # Clear GPU memory after each segment
        clear_gpu_memory()
        
        segment_count += 1
    
    # Unload the image-to-video model
    del i2v_pipe
    clear_gpu_memory()
    
    print(f"Generated {segment_count} segments successfully")
    
    # Now combine the saved frames into a video
    print("Combining frames into video...")
    
    frames_list = []
    for frame_path in all_frame_paths:
        img = cv2.imread(frame_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            frames_list.append(img)
    
    # Ensure we have frames
    if len(frames_list) > 0:
        frames_array = np.array(frames_list)
        
        # Save the final video
        output_path = "pythagorean_theorem_video.mp4"
        export_to_video(frames_array, output_path, fps=fps)
        print(f"Video saved to: {output_path}")
        
        # Display information about the generated video
        print(f"Generated video details:")
        print(f"- Duration: {len(frames_list)/fps:.2f} seconds")
        print(f"- Frames: {len(frames_list)}")
        print(f"- FPS: {fps}")
        print(f"- Resolution: {frames_list[0].shape[0]}x{frames_list[0].shape[1]}")
    else:
        print("No frames were generated or loaded.")

except Exception as e:
    print(f"Error during video generation: {e}")
    import traceback
    traceback.print_exc()

Created 30 visual prompts for video generation
Generating segments one by one...
Loading text-to-image pipeline...


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/608M [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]


Segment 1/30: A right triangle with squares drawn on each side, ...


  0%|          | 0/20 [00:00<?, ?it/s]

Error during video generation: CUDA out of memory. Tried to allocate 180.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 38.12 MiB is free. Process 9962 has 14.70 GiB memory in use. Of the allocated memory 14.24 GiB is allocated by PyTorch, and 342.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Traceback (most recent call last):
  File "/tmp/ipykernel_31/2137607266.py", line 128, in <cell line: 0>
    outputs = i2v_pipe(
              ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/diffusers/pipelines/stable_video_diffusion/pipeline_stable_video_diffusion.py", line 576, in __call__
    noise_pred = self.unet(
                 ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/accelerate/hooks.py", line 170, in new